In [1]:
import os

from salespy import SalesforceClass, EpicorClass

In [2]:
import pandas as pd

In [3]:
# login to Salesforce & get credentials
sf = SalesforceClass(
    username='bko@fs-elliott.com.test', sandbox='test',
    org_id='00D7A0000009Kmw'
)
creds = sf.login()
print(f'--- Logged Into Salesforce ---')

--- Logged Into Salesforce ---


In [4]:
# get Epicor file
directory = (
    '/Users/beekayokay/OneDrive/Projects/'
    'Salesforce Epicor Integration/'
)
file_name = 'test_bookings_df_2.xlsx'
epicor = EpicorClass()
bookings_df = epicor.bookings_to_df(os.path.join(directory, file_name))
print(f'--- Got Epicor Data: {bookings_df.size} ---')

--- Got Epicor Data: 336600 ---


In [5]:
bookings_df.head(5)

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Ship_To_City__c,Ship_To_State__c,Ship_To_Zip__c,Ship_To_Country__c,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c
0,32794,31,754143,2019-11-08,Void,1,SPMX,B1EQ2E290,0.0,4502235891,...,CLAYPOOL,AZ,85532,USA,FSE,AIRSEP,PROGRESS PMT 30,FCA FACTORY,0c9b78d7-3f67-4c2f-a268-4329efe0daa7,32794-31
1,42216,1,752685,2019-10-17,Void,1,PIDE,E211612-1,-4000.0,C14FS0024P,...,VALDIVIA,NaN,0100,CHILE,NaN,INDUS,NET 60 DAYS,FCA FACTORY,bb8ad44d-a9fe-4ae3-8730-5fc5b8611e1d,42216-1
2,48633,1,759073,2020-01-21,Void,1,AP7I,B5EX2E079,-375043.0,13516,...,GREGORY,TX,78359,USA,FSE,INDUS,PROGRESS PMT 90,Ex-Works Export,7762a8f1-bb02-4299-9737-9f4930e30ffb,48633-1
3,49272,14,752651,2019-10-17,New,1,SPMX,B3EU1E117,0.0,4506578294,...,CEDEX,NaN,92227,FRANCE,NaN,AIRSEP,NET 60 DAYS,FCA FACTORY,3ccd1a39-eab7-4809-ba51-f1ad5a14a353,49272-14
4,49272,15,756351,2019-12-11,New,1,SPMX,B3EU1E117,0.0,4506578294,...,CEDEX,NaN,92227,FRANCE,NaN,AIRSEP,NET 60 DAYS,FCA FACTORY,5837a93a-01c5-4256-a573-2b1c5aaaa50e,49272-15


In [6]:
bookings_df['Code_Ref'] = bookings_df['Salesperson__c']
cp_inds = bookings_df['Code_Ref'].str.startswith('D') | bookings_df['Code_Ref'].str.startswith('R') | bookings_df['Code_Ref'].str.startswith('F')
bookings_df.loc[~cp_inds, 'Code_Ref'] = bookings_df.loc[~cp_inds, 'Primary_Salesperson_Origin__c']
cp_inds = bookings_df['Code_Ref'].str.startswith('D') | bookings_df['Code_Ref'].str.startswith('R') | bookings_df['Code_Ref'].str.startswith('F')
bookings_df.loc[~cp_inds, 'Code_Ref'] = bookings_df.loc[~cp_inds, 'Regional_Sales_Manager_Origin__c']
cp_inds = bookings_df['Code_Ref'].str.startswith('D') | bookings_df['Code_Ref'].str.startswith('R') | bookings_df['Code_Ref'].str.startswith('F')
bookings_df.loc[~cp_inds, 'Code_Ref'] = bookings_df.loc[~cp_inds, 'Sales_Representative__c']

In [7]:
code = bookings_df['Code_Ref'].drop_duplicates()
code.dropna(inplace=True)
code_tuple = tuple(code)

In [8]:
acc_df = pd.DataFrame(columns=['Id', 'OwnerId', 'Distributor_Agreement__c', 'Sales_Representative_Agreement__c'])
user_df = pd.DataFrame(columns=['Id', 'Salescode__c'])

In [9]:
acc_temp_list = []
user_temp_list = []

In [10]:
if len(code_tuple) > 0:
    if len(code_tuple) == 1:
        code_tuple = code_tuple[0]

    for num, each in enumerate(code_tuple):
        acc_temp_list.append(each)
        if len(str(acc_temp_list)) < 50000 and num != len(code_tuple)-1:
            continue
        else:
            acc_query = (
                "SELECT Id, OwnerId, Distributor_Agreement__c, Sales_Representative_Agreement__c "
                "FROM Account "
                "WHERE "
                f"(Distributor_Agreement__c IN {tuple(acc_temp_list)} OR "
                f"Sales_Representative_Agreement__c IN {tuple(acc_temp_list)})"
            )
            acc_df_temp = sf.bulk_query(
                creds=creds, object_api='Account',
                query_call=acc_query
            )
            acc_df = acc_df.append(
                acc_df_temp, ignore_index=True
            )
            acc_df_temp = pd.DataFrame()
            acc_temp_list = []

In [11]:
if len(code_tuple) > 0:
    if len(code_tuple) == 1:
        code_tuple = code_tuple[0]

    for num, each in enumerate(code_tuple):
        user_temp_list.append(each)
        if len(str(user_temp_list)) < 50000 and num != len(code_tuple)-1:
            continue
        else:
            user_query = (
                "SELECT Id, Salescode__c "
                "FROM User "
                "WHERE "
                f"(Salescode__c IN {tuple(user_temp_list)} AND "
                "UserType = 'Standard')"
            )
            user_df_temp = sf.bulk_query(
                creds=creds, object_api='User',
                query_call=user_query
            )
            user_df = user_df.append(
                user_df_temp, ignore_index=True
            )
            user_df_temp = pd.DataFrame()
            user_temp_list = []

In [12]:
dist_df = acc_df[['Id', 'OwnerId', 'Distributor_Agreement__c']].copy()
rep_df = acc_df[['Id', 'OwnerId', 'Sales_Representative_Agreement__c']].copy()
dist_df.rename(columns={'Id': 'Channel_Partner__c', 'OwnerId': 'Sales_Lead__c', 'Distributor_Agreement__c': 'Code_Ref'}, inplace=True)
rep_df.rename(columns={'Id': 'CP_Rep', 'OwnerId': 'SL_Rep', 'Sales_Representative_Agreement__c': 'Code_Ref'}, inplace=True)
user_df.rename(columns={'Id': 'SL_User', 'Salescode__c': 'Code_Ref'}, inplace=True)

In [13]:
bookings_df = bookings_df.merge(dist_df, how='left', on='Code_Ref')
bookings_df = bookings_df.merge(rep_df, how='left', on='Code_Ref')
bookings_df = bookings_df.merge(user_df, how='left', on='Code_Ref')

In [14]:
bookings_df.head(5)

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Code_Ref,Channel_Partner__c,Sales_Lead__c,CP_Rep,SL_Rep,SL_User
0,32794,31,754143,2019-11-08,Void,1,SPMX,B1EQ2E290,0.0,4502235891,...,PROGRESS PMT 30,FCA FACTORY,0c9b78d7-3f67-4c2f-a268-4329efe0daa7,32794-31,F113,NaN,NaN,NaN,NaN,NaN
1,42216,1,752685,2019-10-17,Void,1,PIDE,E211612-1,-4000.0,C14FS0024P,...,NET 60 DAYS,FCA FACTORY,bb8ad44d-a9fe-4ae3-8730-5fc5b8611e1d,42216-1,D224,001i000000LRvQPAA1,005i0000006qn6WAAQ,NaN,NaN,NaN
2,48633,1,759073,2020-01-21,Void,1,AP7I,B5EX2E079,-375043.0,13516,...,PROGRESS PMT 90,Ex-Works Export,7762a8f1-bb02-4299-9737-9f4930e30ffb,48633-1,D261,001i000000LRvQaAAL,0050H000009dmdCQAQ,NaN,NaN,NaN
3,49272,14,752651,2019-10-17,New,1,SPMX,B3EU1E117,0.0,4506578294,...,NET 60 DAYS,FCA FACTORY,3ccd1a39-eab7-4809-ba51-f1ad5a14a353,49272-14,R411,NaN,NaN,001i000001RbPbqAAF,005i0000006XnIpAAK,NaN
4,49272,15,756351,2019-12-11,New,1,SPMX,B3EU1E117,0.0,4506578294,...,NET 60 DAYS,FCA FACTORY,5837a93a-01c5-4256-a573-2b1c5aaaa50e,49272-15,R411,NaN,NaN,001i000001RbPbqAAF,005i0000006XnIpAAK,NaN


In [15]:
cp_inds = bookings_df['Channel_Partner__c'].isnull()
bookings_df.loc[cp_inds, 'Channel_Partner__c'] = bookings_df.loc[cp_inds, 'CP_Rep']
sl_inds = bookings_df['Sales_Lead__c'].isnull()
bookings_df.loc[sl_inds, 'Sales_Lead__c'] = bookings_df.loc[sl_inds, 'SL_Rep']
sl_inds = bookings_df['Sales_Lead__c'].isnull()
bookings_df.loc[sl_inds, 'Sales_Lead__c'] = bookings_df.loc[sl_inds, 'SL_User']

In [16]:
bookings_df.head(10)

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Code_Ref,Channel_Partner__c,Sales_Lead__c,CP_Rep,SL_Rep,SL_User
0,32794,31,754143,2019-11-08,Void,1,SPMX,B1EQ2E290,0.0,4502235891,...,PROGRESS PMT 30,FCA FACTORY,0c9b78d7-3f67-4c2f-a268-4329efe0daa7,32794-31,F113,NaN,NaN,NaN,NaN,NaN
1,42216,1,752685,2019-10-17,Void,1,PIDE,E211612-1,-4000.0,C14FS0024P,...,NET 60 DAYS,FCA FACTORY,bb8ad44d-a9fe-4ae3-8730-5fc5b8611e1d,42216-1,D224,001i000000LRvQPAA1,005i0000006qn6WAAQ,NaN,NaN,NaN
2,48633,1,759073,2020-01-21,Void,1,AP7I,B5EX2E079,-375043.0,13516,...,PROGRESS PMT 90,Ex-Works Export,7762a8f1-bb02-4299-9737-9f4930e30ffb,48633-1,D261,001i000000LRvQaAAL,0050H000009dmdCQAQ,NaN,NaN,NaN
3,49272,14,752651,2019-10-17,New,1,SPMX,B3EU1E117,0.0,4506578294,...,NET 60 DAYS,FCA FACTORY,3ccd1a39-eab7-4809-ba51-f1ad5a14a353,49272-14,R411,001i000001RbPbqAAF,005i0000006XnIpAAK,001i000001RbPbqAAF,005i0000006XnIpAAK,NaN
4,49272,15,756351,2019-12-11,New,1,SPMX,B3EU1E117,0.0,4506578294,...,NET 60 DAYS,FCA FACTORY,5837a93a-01c5-4256-a573-2b1c5aaaa50e,49272-15,R411,001i000001RbPbqAAF,005i0000006XnIpAAK,001i000001RbPbqAAF,005i0000006XnIpAAK,NaN
5,52667,2,752213,2019-10-09,Void,8,RDOT,Pack Stk Pts - Ind,0.0,FORECAST MODEL,...,DO NOT MAIL,DDP-1,9dc8b431-9603-43d3-ac3d-2ba6ac023522,52667-2,F110,NaN,005i0000002OS1eAAG,NaN,NaN,005i0000002OS1eAAG
6,52667,5,752214,2019-10-09,Void,5,RDOT,Pack Stk Pts - Ind,0.0,FORECAST MODEL,...,DO NOT MAIL,DDP-1,fb2332fc-ed35-4588-a672-5bd9b812c45b,52667-5,F110,NaN,005i0000002OS1eAAG,NaN,NaN,005i0000002OS1eAAG
7,52667,6,752215,2019-10-09,Void,8,RDOT,Pack Stk Pts - Ind,0.0,FORECAST MODEL,...,DO NOT MAIL,DDP-1,1b3c158c-ad14-4301-a1f9-fa73117b0411,52667-6,F110,NaN,005i0000002OS1eAAG,NaN,NaN,005i0000002OS1eAAG
8,52667,10,752216,2019-10-09,Void,2,RDOT,Pack Stk Pts - Ind,0.0,FORECAST MODEL,...,DO NOT MAIL,DDP-1,397c9cbd-9f8e-42b2-8a68-57e5fc4fe967,52667-10,F110,NaN,005i0000002OS1eAAG,NaN,NaN,005i0000002OS1eAAG
9,53842,7,759240,2020-01-22,New,1,AIFC,B6EL1E142,0.0,4500004343,...,PROGRESS PMT 30,FOB East Coast,087959d1-06d0-454f-8260-e15bf84c29c8,53842-7,R505,001i000000LRvRLAA1,005i0000006qn6WAAQ,001i000000LRvRLAA1,005i0000006qn6WAAQ,NaN


In [17]:
bookings_df = bookings_df.drop(columns=['Code_Ref', 'CP_Rep', 'SL_Rep', 'SL_User'])

In [18]:
bookings_df

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Ship_To_Zip__c,Ship_To_Country__c,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Channel_Partner__c,Sales_Lead__c
0,32794,31,754143,2019-11-08,Void,1,SPMX,B1EQ2E290,0.00,4502235891,...,85532,USA,FSE,AIRSEP,PROGRESS PMT 30,FCA FACTORY,0c9b78d7-3f67-4c2f-a268-4329efe0daa7,32794-31,NaN,NaN
1,42216,1,752685,2019-10-17,Void,1,PIDE,E211612-1,-4000.00,C14FS0024P,...,0100,CHILE,NaN,INDUS,NET 60 DAYS,FCA FACTORY,bb8ad44d-a9fe-4ae3-8730-5fc5b8611e1d,42216-1,001i000000LRvQPAA1,005i0000006qn6WAAQ
2,48633,1,759073,2020-01-21,Void,1,AP7I,B5EX2E079,-375043.00,13516,...,78359,USA,FSE,INDUS,PROGRESS PMT 90,Ex-Works Export,7762a8f1-bb02-4299-9737-9f4930e30ffb,48633-1,001i000000LRvQaAAL,0050H000009dmdCQAQ
3,49272,14,752651,2019-10-17,New,1,SPMX,B3EU1E117,0.00,4506578294,...,92227,FRANCE,NaN,AIRSEP,NET 60 DAYS,FCA FACTORY,3ccd1a39-eab7-4809-ba51-f1ad5a14a353,49272-14,001i000001RbPbqAAF,005i0000006XnIpAAK
4,49272,15,756351,2019-12-11,New,1,SPMX,B3EU1E117,0.00,4506578294,...,92227,FRANCE,NaN,AIRSEP,NET 60 DAYS,FCA FACTORY,5837a93a-01c5-4256-a573-2b1c5aaaa50e,49272-15,001i000001RbPbqAAF,005i0000006XnIpAAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,68348,1,761047,2020-02-28,New,1,PIDE,B5EU2E039-1,1836.78,044061,...,27409,USA,NaN,INDUS,NET 30 DAYS,Ex-Works Export,18c4ec41-6155-4434-8836-8aadaf9a43d8,68348-1,001i000000LRvQ4AAL,0050H00000AFeKeQAL
7656,68349,1,761048,2020-02-28,New,1,PIDA,A8ET2116,70273.32,044062,...,28110,USA,NaN,INDUS,NET 30 DAYS,Ex-Works Export,356044c1-fe09-4cd6-b402-d9321e5f32f4,68349-1,001i000000LRvQ4AAL,0050H00000AFeKeQAL
7657,68349,2,761049,2020-02-28,New,1,PIDE,A8ET2116,70.38,044062,...,28110,USA,NaN,INDUS,NET 30 DAYS,Ex-Works Export,c9831ad8-2614-4633-ab18-87e3d9d29eb8,68349-2,001i000000LRvQ4AAL,0050H00000AFeKeQAL
7658,68349,3,761050,2020-02-28,New,1,PIDE,A8ET2116,70.38,044062,...,28110,USA,NaN,INDUS,NET 30 DAYS,Ex-Works Export,57511a2b-c58f-47f8-978a-e63874d13565,68349-3,001i000000LRvQ4AAL,0050H00000AFeKeQAL
